In [1]:
from dash import Dash, html, dcc, callback, Output, Input, State
import plotly.express as px
import pandas as pd
import geopandas as gpd
import plotly.graph_objects as go
import pycountry
import pycountry_convert
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import dash_ag_grid as dag
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [2]:
trend = pd.read_csv('worldriskindex-trend.csv')

In [3]:
countriesCleaned = pd.read_csv('cleanedCountries.csv')

In [4]:
trend = trend.drop(list(trend.filter(regex='Norm')), axis=1)
trend.columns = trend.columns.str.replace("_Base", "")

In [5]:
meta = pd.read_excel('worldriskindex-meta.xlsx')[['Code', 'Variable']]
rename = dict(zip(meta['Code'], meta['Variable']))

In [6]:
def convert_ISO3_Continent(ISO3):
    try:
        ISO2 = pycountry.countries.get(alpha_3=ISO3).alpha_2
        code = pycountry_convert.country_alpha2_to_continent_code(ISO2)
        continent = pycountry_convert.convert_continent_code_to_continent_name(code)
        return continent
    except:
        return pd.NA

In [7]:
exposureSubCol = ["EI_0" + str(i+1) for i in range(7)]

In [8]:
stats = pd.read_csv('cleanedCountries.csv')
stats = stats.set_index('ISO3')
stats = stats.drop(['Country', 'Region'], axis=1)

In [9]:
def formAggregate(lowerBound):
    exposureSubCol = ["EI_0" + str(i+1) for i in range(7)]
    
    # Filtering to be within bounds
    aggregate = trend[trend['Year'] >= lowerBound]
    
    # Years 
    aggregate = aggregate.groupby(aggregate['ISO3.Code']).agg(
        Risk = ('W', 'mean'),
        Exposure = ('E', 'mean'),
        Vulnerability = ('V', 'mean'))
    
    # Add country and continent
    aggregate['Country'] = aggregate.index.map(lambda x: pycountry.countries.get(alpha_3=x).name)
    aggregate['Continent'] = aggregate.index.map(lambda x: convert_ISO3_Continent(x))
    
    # Add disaster columns
    disasterAggregate = trend.groupby(trend['ISO3.Code'])[exposureSubCol].mean()
    aggregate = aggregate.join(disasterAggregate)

    # Add selected country stats
    aggregate = aggregate.join(stats)

    return aggregate

In [10]:
def generateMap(dataFrame):
    fullFig = px.choropleth(dataFrame, 
                            locations=dataFrame.index, 
                            color="Risk", 
                            hover_name="Country", 
                            hover_data={'Population': True, 
                                        'Area (sq. mi.)': True, 
                                        'GDP ($ per capita)': True, 
                                        'Literacy (%)': True, 
                                        'Phones (per 1000)':True, 
                                        'Birthrate': True, 
                                        'Deathrate': True},
                            color_continuous_scale="Viridis")
    fullFig.update_layout(margin=dict(l=0,r=0,t=0,b=0))
    return fullFig

In [11]:
def generateExposureInd(ISO, dataFrame):
    continent = dataFrame['Continent'][ISO]
    continentDf = dataFrame.loc[dataFrame['Continent'] == continent,:]
    maxExposure = continentDf['Exposure'].max()
    iqrExposure = continentDf['Exposure'].quantile([0.25, 0.5, 0.75])

    fig = go.Figure(go.Indicator(
        mode = "gauge+number",
        value = dataFrame['Exposure'][ISO],
        gauge = {'bar': {'color': 'black'},
             'axis': {'range': [0, maxExposure]},
             'steps': [
                 {'range': [0, iqrExposure[0.25]], 'color': "green"},
                 {'range': [iqrExposure[0.25], iqrExposure[0.5]], 'color': "yellow"},
                 {'range': [iqrExposure[0.5], iqrExposure[0.75]], 'color': "orange"},
                 {'range': [iqrExposure[0.75], maxExposure], 'color': "red"}
             ]
             }
        ),
        layout = {
            'title': 'Exposure Indicator',
        }
    )
    return fig

def generateVulnerabilityInd(ISO, dataFrame):
    continent = dataFrame['Continent'][ISO]
    continentDf = dataFrame.loc[dataFrame['Continent'] == continent,:]
    maxVulnerability = continentDf['Vulnerability'].max()
    iqrVulnerability = continentDf['Vulnerability'].quantile([0.25, 0.5, 0.75])

    fig = go.Figure(go.Indicator(
        mode = "gauge+number",
        value = dataFrame['Vulnerability'][ISO],
        gauge = {'bar': {'color': 'black'},
             'axis': {'range': [0, maxVulnerability]},
             'steps': [
                 {'range': [0, iqrVulnerability[0.25]], 'color': "green"},
                 {'range': [iqrVulnerability[0.25], iqrVulnerability[0.5]], 'color': "yellow"},
                 {'range': [iqrVulnerability[0.5], iqrVulnerability[0.75]], 'color': "orange"},
                 {'range': [iqrVulnerability[0.75], maxVulnerability], 'color': "red"}
             ]
             }
        ),
        layout = {
            'title': 'Vulnerability Indicator',
        }
    )
    return fig

In [12]:
def generateDisasterGraph(countryCode, dataFrame):
    countryExp = dataFrame.loc[countryCode, exposureSubCol].astype('float64')
    continentExp = dataFrame.loc[dataFrame['Continent'] == dataFrame.loc[countryCode, 'Continent'], exposureSubCol].mean()
    exposure = pd.DataFrame({
        'country': countryExp,
        'continent': continentExp
    })
    exposure['disaster'] = exposure.index.map(lambda x: rename[x])
    
    fig = px.bar(exposure, x='disaster', y=['country', 'continent'], barmode='overlay')
    return fig



In [13]:
continents = {
    "North America" : (55, -105),
    "South America" : (-9,-56),
    "Asia" : (34, 101),
    "Africa" : (9, 35),
    "Europe" : (55, 15),
    "Oceania" : (-23, 140)
}

Mason's Segment(?)

TODO:
 - Dropdown for Vulnerability Type, Continent, Country DONE
 - Line graph indicating change in vulnerability (Time series) DONE
    - Prediction for future vulnerability? DONE
 - Top/Bottom 3 countries for the statistic DONE
 - Most correlated statistics for said vulnerability DONE

Copy of risk index with only the countries, years, and exposures

In [14]:
def subcategories(searchCategories, lengthOfCategories):
    fixedSubstring = "_"
    res = []

    for i in range(len(searchCategories)):
        category = searchCategories[i]
        length = lengthOfCategories[i]

        for j in range(length):
            lengthFormater = ""

            if (j < 9):
                lengthFormater = "0"

            subCategory = category + fixedSubstring + lengthFormater + str(j+1)
            res.append(subCategory)
    
    return res

In [15]:
search = ["S", "C", "A"]
categoryLength = [5, 3, 3]

vul = subcategories(search, categoryLength)

In [16]:
trend_exposure = trend[['WRI.Country', 'ISO3.Code', 'Year'] + vul]

Line graph for specific vulnerability in select country

In [17]:
def vulnChart(vuln, ISO):
    df_temp = trend[['WRI.Country', 'ISO3.Code', 'Year', vuln]]
    df_temp = df_temp.loc[df_temp['ISO3.Code'] == ISO]
    
    fig = px.line(df_temp, x='Year', y=vuln, title=f'{rename[vuln]} Risk')
    fig.update_traces(mode='markers+lines')
    return fig

Find most and least correlated statistics

In [18]:
recentTrend = trend[trend['Year'] >= 2025]
recentTrend = recentTrend[['ISO3.Code', 'W', 'E', 'V']]

In [19]:
cloneCountries = countriesCleaned.copy()
recentTrend.rename(columns={'ISO3.Code': 'ISO3'}, inplace=True)

merged = recentTrend.merge(cloneCountries, on='ISO3', how='inner')

In [20]:
mergedClone = merged[['ISO3','Country', 'V', 'Infant mortality (per 1000 births)', 'Phones (per 1000)', 'Area (sq. mi.)','Literacy (%)', 'Agriculture']]

top5_2025 = mergedClone.nlargest(5, 'V')
bot5_2025 = mergedClone.nsmallest(5, 'V')

topbot5_2025 = pd.concat([top5_2025, bot5_2025])

In [21]:
def mergedlookup(ISO3):
    return mergedClone.loc[mergedClone['ISO3'] == ISO3]

In [22]:
train_range = ('2000-01-31', '2025-12-31')
forecast_range = ('2000-01-31', '2030-12-31')
test_range = ('2026-01-31', '2030-12-31')

forecast_length = len(pd.date_range(start=forecast_range[0], end=forecast_range[1], freq='YE'))

trend_datetime = trend.copy()
trend_datetime['Date'] = pd.to_datetime(trend_datetime['Year'].astype(str) + '-01-31')

In [23]:
def generateHoltForecast(ISO3):
    target = trend_datetime.loc[trend_datetime['ISO3.Code'] == ISO3, ['Date', 'V']]
    
    target.set_index('Date', inplace=True)
    
    holt_model = ExponentialSmoothing(
        target, trend='add', seasonal=None, damped_trend = True, initialization_method="estimated"
    ).fit()
    
    holt_forecast = holt_model.forecast(forecast_length)
    holt_test = holt_forecast.loc[test_range[0]:test_range[1]]
    return holt_test

In [24]:
def generatePredictedVulnerabilityGraph(ISO3):
    holt_forecast = generateHoltForecast(ISO3)
    
    fig = px.line(holt_forecast, x=holt_forecast.index, y=holt_forecast.values, title='Predicted Vulnerability (Holt\'s Linear Trend Model with Damping)')
    fig.update_traces(mode='markers+lines')
    fig.update_xaxes(title='Year')
    fig.update_yaxes(title='Vulnerability Index')
    return fig

In [25]:
isoNames = trend.loc[:,['ISO3.Code','WRI.Country']].drop_duplicates(subset='ISO3.Code')
isoToName = dict(zip(isoNames['ISO3.Code'], isoNames['WRI.Country']))

In [26]:
x = ['Infant mortality (per 1000 births)', 'Phones (per 1000)', 'Area (sq. mi.)', 'Literacy (%)', 'Agriculture']
y = 'Vulnerability'
data = formAggregate(2000).dropna()
X = data.loc[:, x]
Y = data[y]

mdl = linear_model.LinearRegression().fit(X, Y)
print(mdl.score(X, Y))
print(mdl.coef_)

# mdl.predict(input)

0.6194223896232861
[ 9.54184646e-02 -1.70281462e-02  1.41290479e-06 -1.73957581e-01
  2.38420058e+01]


In [27]:
app = Dash(__name__)

app.layout = [
    html.Div(
        style={'display': 'flex'},
        children=[
            html.Div(
                children=[
                    html.Div(
                        style={'display': 'flex'},
                        children=[
                            dcc.Dropdown(id='continents', 
                                        options=[{'label': c, 'value': c} for c in continents], 
                                        value='North America', 
                                        style={'width': '25vw', 'height': '5vh'}),
                            dcc.Dropdown(id='aggregate',
                                        options=[{'label': 'Last 25 years', 'value':2000},
                                                {'label': 'Last 10 years', 'value':2015},
                                                {'label': 'Last 5 years', 'value':2020}],
                                        value=2000,
                                        style={'width': '25vw', 'height': '5vh'})
                        ]
                    ),
                    
                    dcc.Store(id='iso'),
                    dcc.Store(id='dataFrame', data=formAggregate(2000).to_dict()),
                    dcc.Graph(id='map', style={'width': '50vw', 'height': '35vh'}, responsive=True),

                    html.Div(
                        style={'display': 'flex'},
                        children=[
                            dcc.Graph(id='exposure', style={'width': '25vw', 'height': '25vh'}, responsive=True),
                            dcc.Graph(id='vulerability', style={'width': '25vw', 'height': '25vh'}, responsive=True)
                        ]
                    ),

                    dcc.Graph(id='disaster', style={'width': '50vw', 'height': '35vh'}, responsive=True)
                ]
            ),

            html.Div(
                children=[
                    html.Div(className='row', style={'display': 'flex'}, children=[
                        dcc.Dropdown(id='disasterType', options=[{'label': rename[col], 'value': col} for col in vul], value='S_01', style={'width': '25vw', 'height': '5vh'}),
                        dcc.Dropdown(id='disasterCountry', options=[{'label': isoToName[iso], 'value': iso} for iso in isoToName], value='CAN', style={'width': '25vw', 'height': '5vh'})]
                    ),

                    dcc.Graph(id='disasterCountryGraph', figure={}, style={'width': '50vw', 'height': '32.5vh'}, responsive=True),
                    
                    dcc.Graph(id='predictedVulnerabilityGraph', figure={}, style={'width': '50vw', 'height': '32.5vh'}, responsive=True),
                    
                    html.Div(style={'display': 'flex'}, children=[
                        dag.AgGrid(id='countryData', columnDefs=[{'headerName': col, 'field': col} for col in topbot5_2025.columns], style={'width': '25vw', 'height': '20vh'}),
                        dag.AgGrid(rowData=topbot5_2025.to_dict('records'), columnDefs=[{'headerName': col, 'field': col} for col in topbot5_2025.columns], style={'width': '25vw', 'height': '20vh'})    
                    ]),

                    html.Div(children=[
                        html.Div(style={'display': 'flex'}, children=[
                            dcc.Input(id='infant_mortality', type='number', placeholder='Infant Mortality', style={'width': '9.5vw', 'height': '5vh'}),
                            dcc.Input(id='phones', type='number', placeholder='Phones per 1000', style={'width': '9.5vw', 'height': '5vh'}),
                            dcc.Input(id='area', type='number', placeholder='Area', style={'width': '9.5vw', 'height': '5vh'}),
                            dcc.Input(id='literacy', type='number', placeholder='Literacy %', style={'width': '9.5vw', 'height': '5vh'}),
                            dcc.Input(id='agriculture', type='number', placeholder='Agriculture Industry %', style={'width': '9.5vw', 'height': '5vh'})
                        ]),
                        html.Div(style={'display': 'flex'}, children=[
                            html.Div(id='vulnerability_output', children="Predicted Vulnerabilty:", style={'width': '20vw', 'height': '5vh'}),
                            html.Button('Predict', id='prediction_button', n_clicks=0, style={'width': '10vw', 'height': '5vh'})
                        ])
                    ])
                ]
            )
        ]
    ) 
]

@app.callback(
    Output('dataFrame', 'data'),
    Input('aggregate', 'value')
)
def generateDataFrame(bound):
    return formAggregate(bound).to_dict()

@app.callback(
        Output('iso', 'data'),
        Input('map', 'clickData')
)
def getISOCode(clickData):
    if clickData:
        ISO = clickData['points'][0]['location']
        return ISO
    return 'CAN'

@app.callback(
    Output('map', 'figure'),
    Input('continents', 'value'),
    Input('dataFrame', 'data')
)
def getMap(selected, data):
    df = pd.DataFrame.from_dict(data)
    values = continents[selected]
    fig = generateMap(df)

    fig.update_geos(
        center_lat=values[0],
        center_lon=values[1],
        projection_scale=3
    )

    return fig

@app.callback(
    Output('exposure', 'figure'),
    Input('iso', 'data'),
    Input('dataFrame', 'data')
)
def getExposureInd(ISO, data):
    df = pd.DataFrame.from_dict(data)
    return generateExposureInd(ISO, df)
    
@app.callback(
    Output('vulerability', 'figure'),
    Input('iso', 'data'),
    Input('dataFrame', 'data')
)
def getVulnerabilityInd(ISO, data):
    df = pd.DataFrame.from_dict(data)
    return generateVulnerabilityInd(ISO, df)

@app.callback(
    Output('disaster', 'figure'),
    Input('iso', 'data'),
    Input('dataFrame', 'data')
)
def getDisasterGraph(ISO, data):
    df = pd.DataFrame.from_dict(data)
    return generateDisasterGraph(ISO, df)
    
        
@app.callback(
    Output('disasterCountryGraph', 'figure'),
    Input('disasterCountry', 'value'),
    Input('disasterType', 'value')
)
def getDisasterCountryGraph(countryISO, disasterType):
    return vulnChart(disasterType, countryISO)

@app.callback(
    Output('countryData', 'rowData'),
    Input('disasterCountry', 'value')
)
def updateCountryData(countryISO):
    data = mergedlookup(countryISO)
    return data.to_dict('records')

@app.callback(
    Output('predictedVulnerabilityGraph', 'figure'),
    Input('disasterCountry', 'value')
)
def getPredictedVulnerabilityGraph(countryISO):
    return generatePredictedVulnerabilityGraph(countryISO)

@app.callback(
    Output('vulnerability_output', 'children'),
    Input('prediction_button', 'n_clicks'),
    [State("{}".format(pred), "value") for pred in ['infant_mortality', 'phones', 'area', 'literacy', 'agriculture']],
    prevent_initial_call=True
)
def update_output(n_clicks, *vals):
    for v in vals:
        if v == None:
            return 'Predicted Vulnerability:'
        
    return f'Predicted Vulnerability: {mdl.predict(pd.DataFrame(dict(zip(x, vals)), index=[0]))}'

if __name__ == '__main__':
    #app.run(debug=True)
    app.run(jupyter_mode='external')


Dash app running on http://127.0.0.1:8050/


/home/user/anaconda3/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency YE-JAN will be used.

